In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [6]:
data = pd.read_csv('./Week-7-MachineLearning/weather/daily_weather.csv')
data.head(3)

,number,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,relative_humidity_9am,relative_humidity_3pm
0,0,918.060000,74.822000,271.100000,2.080354,295.400000,2.863283,0.0,0.0,42.420000,36.160000
1,1,917.347688,71.403843,101.935179,2.443009,140.471548,3.533324,0.0,0.0,24.328697,19.426597
2,2,923.040000,60.638000,51.000000,17.067852,63.700000,22.100967,0.0,20.0,8.900000,14.460000


In [7]:
data.columns

Index(['number', 'air_pressure_9am', 'air_temp_9am', 'avg_wind_direction_9am',
       'avg_wind_speed_9am', 'max_wind_direction_9am', 'max_wind_speed_9am',
       'rain_accumulation_9am', 'rain_duration_9am', 'relative_humidity_9am',
       'relative_humidity_3pm'],
      dtype='object')

In [14]:
print(len(data[data.isnull().any(axis=1)]))
print(len(data))

31
1095


In [16]:
before_drop = data.shape
data = data.dropna()
after_drop = data.shape
print(before_drop, after_drop)

(1095, 11) (1064, 11)


In [17]:
data.relative_humidity_3pm.describe()

count    1064.000000
mean       35.148381
std        22.365475
min         5.300000
25%        17.360468
50%        24.371286
75%        51.922500
max        92.250000
Name: relative_humidity_3pm, dtype: float64

In [19]:
HIGH_HUMID = data.relative_humidity_3pm.median()
HIGH_HUMID

24.371285775232927

In [20]:
len(data[data.relative_humidity_3pm > HIGH_HUMID])

532

In [21]:
len(data[data.relative_humidity_3pm > 24.99])

529

In [22]:
clean_data = data.copy()

In [25]:
clean_data['high_humidity_label'] = (clean_data['relative_humidity_3pm'] > HIGH_HUMID) *1
print(clean_data.high_humidity_label[0:5])

0    1
1    0
2    0
3    0
4    1
Name: high_humidity_label, dtype: int64


In [26]:
y = clean_data[['high_humidity_label']].copy()
y.head(3)

,high_humidity_label
0,1
1,0
2,0


In [28]:
col_name = clean_data.columns.tolist()
col_name

['number',
 'air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am',
 'relative_humidity_9am',
 'relative_humidity_3pm',
 'high_humidity_label']

In [29]:
col_name.remove('number')
col_name.remove('relative_humidity_9am')
col_name.remove('relative_humidity_3pm')
col_name.remove('high_humidity_label')
col_name

['air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am']

In [30]:
X = clean_data[col_name].copy()
X.head(3)

,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am
0,918.060000,74.822000,271.100000,2.080354,295.400000,2.863283,0.0,0.0
1,917.347688,71.403843,101.935179,2.443009,140.471548,3.533324,0.0,0.0
2,923.040000,60.638000,51.000000,17.067852,63.700000,22.100967,0.0,20.0


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=324)

In [33]:
humidity_classifier = DecisionTreeClassifier(max_leaf_nodes=10, random_state=0)
humidity_classifier.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=10,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [34]:
predictions = humidity_classifier.predict(X_test)

In [35]:
predictions[:10]

array([0, 0, 1, 1, 1, 1, 0, 0, 0, 1])

In [37]:
y_test.high_humidity_label.iloc[:10]

456     0
845     0
693     1
259     1
723     1
224     1
300     1
442     0
585     1
1057    1
Name: high_humidity_label, dtype: int64

In [38]:
accuracy_score(y_true=y_test, y_pred=predictions)

0.8125

# Regression

In [1]:
import sqlite3
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
cnx = sqlite3.connect('./Week-7-MachineLearning/Big/database.sqlite')
df = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)

In [3]:
df.head(3)

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0


In [4]:
df.shape

(183978, 42)

In [5]:
len(df.id.unique())

183978

In [17]:
col_name = df.columns.tolist()
col_name[0:9]

['id',
 'player_fifa_api_id',
 'player_api_id',
 'date',
 'overall_rating',
 'potential',
 'preferred_foot',
 'attacking_work_rate',
 'defensive_work_rate']

In [13]:
help(col_name.pop)

Help on built-in function pop:

pop(...) method of builtins.list instance
    L.pop([index]) -> item -- remove and return item at index (default last).
    Raises IndexError if list is empty or index is out of range.



In [18]:
col_name.remove('id')
col_name.remove('player_fifa_api_id')
col_name.remove('player_api_id')
col_name.remove('date')
col_name.remove('overall_rating')
col_name.remove('preferred_foot')
col_name.remove('attacking_work_rate')
col_name.remove('defensive_work_rate')

col_name

['potential',
 'crossing',
 'finishing',
 'heading_accuracy',
 'short_passing',
 'volleys',
 'dribbling',
 'curve',
 'free_kick_accuracy',
 'long_passing',
 'ball_control',
 'acceleration',
 'sprint_speed',
 'agility',
 'reactions',
 'balance',
 'shot_power',
 'jumping',
 'stamina',
 'strength',
 'long_shots',
 'aggression',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes']

In [21]:
features = col_name.copy()

In [20]:
target = ['overall_rating']

In [22]:
df = df.dropna()

In [23]:
X = df[features]
y = df[target]

In [27]:
#X.iloc[2]
y.iloc[2]

overall_rating    62.0
Name: 2, dtype: float64

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                   random_state=324)

In [30]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [31]:
y_prediction = regressor.predict(X_test)
y_prediction[0:10]

array([[66.52174582],
       [79.78338402],
       [66.58730221],
       [74.99483325],
       [66.2076076 ],
       [62.78974237],
       [76.23216082],
       [69.37084598],
       [66.91973037],
       [73.99451949]])

In [32]:
y_test.describe()

,overall_rating
count,54107.000000
mean,68.630972
std,7.042143
min,33.000000
25%,64.000000
50%,69.000000
75%,73.000000
max,94.000000


In [33]:
RMSE = sqrt(mean_squared_error(y_true=y_test, y_pred=y_prediction))

In [34]:
print(RMSE)

2.7991799596584794


In [35]:
tree_regressor = DecisionTreeRegressor(max_depth=20)
tree_regressor.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=20, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [37]:
y_prediction2 = tree_regressor.predict(X_test)
y_prediction2

array([64.        , 84.        , 62.16666667, ..., 60.        ,
       61.28571429, 73.        ])

In [39]:
y_test.describe()

,overall_rating
count,54107.000000
mean,68.630972
std,7.042143
min,33.000000
25%,64.000000
50%,69.000000
75%,73.000000
max,94.000000


In [40]:
RMSE2 = sqrt(mean_squared_error(y_true=y_test, y_pred=y_prediction2))

In [41]:
print(RMSE2)

1.4417636383310566


In [46]:
print(y_test.values[0:5])
print(y_prediction[0:5])
print(y_prediction2[0:5])

[[66.]
 [83.]
 [65.]
 [74.]
 [65.]]
[[66.52174582]
 [79.78338402]
 [66.58730221]
 [74.99483325]
 [66.2076076 ]]
[64.         84.         62.16666667 75.         65.        ]


In [63]:
#y_test.values[:, 0]
y_prediction[:,0]
y_prediction2.shape

(54107,)

In [64]:
result = pd.DataFrame({'True Val':y_test.values[:,0], 
                       'Linear Val': y_prediction[:,0],
                      'Decision Val': y_prediction2[:]})

In [65]:
result.head(5)

,Decision Val,Linear Val,True Val
0,64.000000,66.521746,66.0
1,84.000000,79.783384,83.0
2,62.166667,66.587302,65.0
3,75.000000,74.994833,74.0
4,65.000000,66.207608,65.0


In [66]:
len(y_test)

54107

In [68]:
result.iloc[5000]

Decision Val    63.000000
Linear Val      63.340685
True Val        63.000000
Name: 5000, dtype: float64